<a href="https://colab.research.google.com/github/raagKush/ML_learning_google_collab/blob/main/autoencoders_denoise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from tensorflow.keras.layers import Conv2D, MaxPooling2D, UpSampling2D

NOISE = 0.5
(x_train,_),(x_test,_)= tf.keras.datasets.mnist.load_data()

x_train_norm = x_train/255.
x_test_norm = x_test/255.

x_train_norm_reshaped = x_train_norm.reshape(len(x_train_norm),28,28,1)
x_test_norm_reshaped = x_test_norm.reshape(len(x_test_norm),28,28,1)

x_train_noisy = x_train_norm + NOISE*np.random.normal(loc = 0.0, scale = 1.0, size = x_train.shape)
x_test_noisy = x_test_norm + NOISE*np.random.normal(loc = 0.0, scale = 1.0, size = x_test.shape)

x_train_clipped = np.clip(x_train_noisy,0.,1.)
x_test_clipped = np.clip(x_test_noisy,0.,1.)

plt.figure()
plt.imshow(x_test_clipped[0])

x_train_reshaped = x_train_clipped.reshape(len(x_train_clipped),28,28,1)
x_test_reshaped = x_test_clipped.reshape(len(x_test_clipped),28,28,1)

model = tf.keras.models.Sequential()

model.add(Conv2D(32,(3,3), activation ='relu',input_shape=(28,28,1),padding='same'))
model.add(MaxPooling2D((2,2),padding = 'same'))
model.add(Conv2D(16,(3,3),activation= 'relu', padding = 'same'))
model.add(MaxPooling2D((2,2),padding = 'same'))
model.add(Conv2D(8,(3,3),activation = 'relu', padding = 'same'))
model.add(MaxPooling2D((2,2),padding = 'same'))

model.add(Conv2D(8,(3,3),activation='relu',padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(16,(3,3),activation='relu',padding = 'same'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(32,(3,3),activation='relu'))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(1,(3,3),activation='relu',padding = 'same'))

model.summary()

model.compile(optimizer='adam',loss='mean_squared_error')

model.fit(x_train_reshaped,x_train_norm_reshaped,batch_size=500,epochs=10,validation_split=0.1)

preds = model.predict(x_test_reshaped)

plt.figure()
plt.imshow(x_test_reshaped[10]*255)

plt.figure()
plt.imshow(preds[10]*255)
